In [1]:
import numpy as np
import random
import pickle

In [2]:
# loading training set features
f = open("Datasets/train_set_features.pkl", "rb")
train_set_features2 = pickle.load(f)
f.close()

# reducing feature vector length 
features_STDs = np.std(a=train_set_features2, axis=0)
train_set_features = train_set_features2[:, features_STDs > 52.3]

# changing the range of data between 0 and 1
train_set_features = np.divide(train_set_features, train_set_features.max())

# loading training set labels
f = open("Datasets/train_set_labels.pkl", "rb")
train_set_labels = pickle.load(f)
f.close()

# ------------
# loading test set features
f = open("Datasets/test_set_features.pkl", "rb")
test_set_features2 = pickle.load(f)
f.close()

# reducing feature vector length 
features_STDs = np.std(a=test_set_features2, axis=0)
test_set_features = test_set_features2[:, features_STDs > 48]

# changing the range of data between 0 and 1
test_set_features = np.divide(test_set_features, test_set_features.max())

# loading test set labels
f = open("Datasets/test_set_labels.pkl", "rb")
test_set_labels = pickle.load(f)
f.close()

# ------------
# preparing our training and test sets - joining datasets and lables
train_set = []
test_set = []

for i in range(len(train_set_features)):
    label = np.array([0,0,0,0])
    label[int(train_set_labels[i])] = 1
    label = label.reshape(4,1)
    train_set.append((train_set_features[i].reshape(102,1), label))
    

for i in range(len(test_set_features)):
    label = np.array([0,0,0,0])
    label[int(test_set_labels[i])] = 1
    label = label.reshape(4,1)
    test_set.append((test_set_features[i].reshape(102,1), label))

# shuffle
random.shuffle(train_set)
random.shuffle(test_set)

In [3]:
def sigmoid(z):
    return 1/(1 + np.exp(-z, dtype=np.longdouble))

def d_sigmoid(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [7]:
class MLP:
    
    weights: list[np.ndarray]
    
    def __init__(self, layers_size: list[tuple[int, int]]):
        self.weights = []
        self.biases = []
        self.learning_rate = 1
        
        for size in layers_size:
            self.weights.append(np.random.normal(size=size))
            self.biases.append(np.zeros((1, size[1])).astype(np.longdouble))
        
        self.layers_number = len(self.weights)

    def feed(self, data):
        W = self.weights
        b = self.biases
        
        result = []
        o = sigmoid(W[0].T.dot(data) + b[0])
        result.append(o)
        for i in range(1, self.layers_number):
            o = sigmoid(W[i].T.dot(o) + b[i])
            result.append(o)
        return result
    
    def train(x, y, batch_size=10, epoch_number=10):
        w = self.weights
        b = self.biases
        for i in range(epoch_number):
            for j in range(0, x.shape[0], batch_size):
                data = x[j:j+batch_size]
                y_ex = y[j:j+batch_size]       # Expected Output
                out = self.feed(data)
                
                # 
                e3 = (out[3] - y_ex) * d_sigmoid(out[3])
                
                e2 = e3.dot(w[3]) * d_sigmoid(out[2])
                
                e1 = e2.dot(w[2]) * d_sigmoid(out[1])
                
                w[1] -= self.learning_rate * (1/batch_size) * data.T.dot(e1)
                w[2] -= self.learning_rate * (1/batch_size) * out[1].T.dot(e1)
                w[3] -= self.learning_rate * (1/batch_size) * out[2].T.dot(e1)
                
        pass
        

    
mlp = MLP([(102, 150), (150, 60), (60, 4)])
a = mlp.feed(np.ones((102, 1)))


TypeError: RandomState.shuffle() takes exactly one argument (2 given)